In [19]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [20]:
# Path to chromedriver
!which chromedriver

which: no chromedriver in (/c/Users/emily/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/emily/bin:/c/Windows/system32:/c/Windows:/c/Windows/System32/Wbem:/c/Windows/System32/WindowsPowerShell/v1.0:/c/Windows/System32/OpenSSH:/cmd:/c/Program Files/MongoDB/Server/4.4/bin:/c/Users/emily/anaconda3:/c/Users/emily/anaconda3/Library/mingw-w64/bin:/c/Users/emily/anaconda3/Library/usr/bin:/c/Users/emily/anaconda3/Library/bin:/c/Users/emily/anaconda3/Scripts:/c/Users/emily/AppData/Local/Microsoft/WindowsApps:/c/Users/emily/AppData/Local/Programs/Microsoft VS Code/bin:/usr/bin/vendor_perl:/usr/bin/core_perl)


In [21]:
# Set the executable path and initialize the chrome browser in splinter
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\emily\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [22]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [23]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [24]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8805/moxie-could-help-future-rockets-launch-off-mars/" target="_self">MOXIE Could Help Future Rockets Launch Off Mars</a></div>

In [25]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [26]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

### JPL Space Images Featured Image

In [27]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [28]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [29]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [30]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [31]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA19404_hires.jpg'

In [32]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19404_hires.jpg'

### Mars Facts

In [33]:
df = pd.read_html('http://space-facts.com/mars/')[0]

df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [34]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

### Mars Weather

In [36]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [37]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [38]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

<table class="mb_table" id="weather_observation" style="width:100%;">
 <thead>
  <tr>
   <th colspan="2" scope="col">
    Time
   </th>
   <th colspan="3" id="temperature_lbl" scope="col">
    Air Temperature (
    <span class="lbl_fahrenheit">
     °F
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_celsius fadeBlack">
     °C
    </span>
    )
   </th>
   <th colspan="4" id="windspeed_lbl" scope="col">
    Wind Speed (
    <span class="lbl_mph">
     mph
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_mps fadeBlack">
     m/s
    </span>
    )
   </th>
   <th colspan="3" id="pressure_lbl" scope="col">
    Pressure (Pa)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr id="weather_top">
   <th class="sol" scope="row">
    Date
   </th>
   <th class="sol" scope="row">
    Sol
   </th>
   <td class="temperature max">
    Max.
   </td>
   <td class="temperature avg">
    Avg.
   </td>
   <td class="temperature min">
    Min.
   </td>
   <t

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [ ]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [ ]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

In [49]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
img_soup = soup(html, 'html.parser')
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url = f'https://astrogeology.usgs.gov{img_url_rel}'
img_url

# -------

def hemisphere_img_urls():
    # Initiate headless driver for deployment
    browser = Browser("chrome", executable_path="chromedriver", headless=True)
    news_title, news_paragraph = mars_news(browser)
    # Run all scraping functions and store results in dictionary
    data = {
      "hemis_title": hemis_title,
      "hemis_url": hemis_url
    }
    browser.quit()
    return data

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=50915): Max retries exceeded with url: /session/52d3ba78acea3d1b93fd6969e5fd0a4e/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001469F874370>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
hemisphere_image_urls = img_soup.select_one('figure.lede a img').get("src")

hemi_titles = hemis_soup.find('h3').text

tags_box = html_soup.find('div', class_='tags-box')
tags = tags_box.find_all('a', class_='tag')
for tag in tags:
    word= tag.text
    print(word)

In [41]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[]

In [42]:
# 5. Quit the browser
browser.quit()